# **1. Using Huggingface**

In [ ]:
!pip install -U transformers
!pip install -U huggingface_hub

In [ ]:
from huggingface_hub import login
login()

In [ ]:
import requests
import torch
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor

model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"

model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
processor = AutoProcessor.from_pretrained(model_id)

url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/0052a70beed5bf71b92610a43a52df6d286cd5f3/diffusers/rabbit.jpg"
image = Image.open(requests.get(url, stream=True).raw)

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": "If I had to write a haiku for this one, it would be: "}
    ]}
]
input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt"
).to(model.device)

output = model.generate(**inputs, max_new_tokens=30)
print(processor.decode(output[0]))


# **2. GROQ**

In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00


In [ ]:
import os

from groq import Groq

client = Groq(
    api_key=" ",
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.2-11b-vision-preview",
)

print(chat_completion.choices[0].message.content)



Fast language models have become increasingly important in recent years due to their ability to process and generate large amounts of text quickly, accurately, and efficiently. Here are some reasons why fast language models are significant:

1. **Scalability**: Fast language models can handle a large volume of text data, making them suitable for applications that require processing massive amounts of information, such as language translation, text summarization, and text classification.
2. **Real-time applications**: Fast language models enable real-time text processing, which is essential for applications like chatbots, virtual assistants, and voice-to-text systems. They can quickly respond to user queries, providing fast and accurate results.
3. **Research and development**: Fast language models facilitate rapid testing and evaluation of new language processing techniques, models, and algorithms. This accelerates the development of natural language processing (NLP) technologies, enab

In [ ]:
# LLAMA 3.2 with image input:

completion = client.chat.completions.create(
    model="llama-3.2-11b-vision-preview",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What's in this image?"
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://img.buzzfeed.com/buzzfeed-static/static/2018-07/16/16/tmp/buzzfeed-prod-web-01/tmp-name-2-27495-1531773764-5_dblbig.jpg?resize=1200:*"
                    }
                }
            ]
        },
        {
            "role": "assistant",
            "content": ""
        }
    ],
    temperature=1,
    max_tokens=1024,
    top_p=1,
    stream=False,
    stop=None,
)

print(completion.choices[0].message.content)


This image appears to capture several extremely cute and playful panda bear cubs engaging in a game of "follow the leader" on a wooden floor.

The cubs, sporting distinctive black and white fur, are scattered across the image. Some are lying on their stomachs with all four paws in the air, while others are resting on their sides. Each cub seems to be focused on something on the floor.

Near the bottom right corner, two blocks of different sizes sit on the floor. The cubs are mostly ignoring them, but it seems likely they were used as toys during the game of "follow the leader".
